In [55]:
!pip install mediapipe==0.9.0.1 opencv-python==4.7.0.68 pandas scikit-learn==1.0.2 pyttsx3

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Live View

In [9]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Live Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Capturing Landmarks & Exporting to CSV

In [10]:
import csv
import os
import numpy as np

In [11]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [13]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [45]:
with open('coords_class.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [46]:
class_name = "Hand Pain"

In [47]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords_class.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow(str(class_name), image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Training a custom model

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [48]:
df = pd.read_csv('coords.csv')

In [49]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.466250,0.331764,-0.907347,0.999906,0.488859,0.275133,-0.851194,0.999823,0.501955,...,0.001799,0,0.519976,0.273510,0.020471,0,0.524371,0.268968,0.021337,0
1,Normal,0.470933,0.328804,-0.778751,0.999902,0.490143,0.273135,-0.720188,0.999819,0.502533,...,0.002995,0,0.525391,0.273477,0.023576,0,0.529398,0.270235,0.024468,0
2,Normal,0.473655,0.327398,-0.766039,0.999898,0.490900,0.272356,-0.707355,0.999816,0.502857,...,0.002160,0,0.524868,0.273799,0.021655,0,0.528982,0.270436,0.022443,0
3,Normal,0.475893,0.325781,-0.764224,0.999896,0.492181,0.271684,-0.705913,0.999815,0.503827,...,0.003089,0,0.524431,0.274338,0.022729,0,0.528567,0.270888,0.023579,0
4,Normal,0.476316,0.324335,-0.698947,0.999903,0.492545,0.270901,-0.643586,0.999828,0.504012,...,0.003550,0,0.522296,0.274781,0.022839,0,0.526642,0.270979,0.023696,0


In [50]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
979,Hand Pain,0.539917,0.300630,-0.519261,0.999859,0.560269,0.252916,-0.474520,0.999697,0.574547,...,0.002297,0,0.581374,0.257538,0.012183,0,0.585121,0.252797,0.012560,0
980,Hand Pain,0.538935,0.301239,-0.503000,0.999870,0.559504,0.252953,-0.459029,0.999721,0.574010,...,0.002378,0,0.582757,0.257123,0.012633,0,0.586475,0.252842,0.013027,0
981,Hand Pain,0.539426,0.301248,-0.581417,0.999878,0.560021,0.252938,-0.533026,0.999737,0.574460,...,0.003969,0,0.584337,0.254576,0.014584,0,0.588096,0.249696,0.015153,0
982,Hand Pain,0.539693,0.301557,-0.641233,0.999886,0.560967,0.252967,-0.595796,0.999753,0.575460,...,0.003809,0,0.585066,0.252302,0.014961,0,0.588834,0.247320,0.015571,0
983,Hand Pain,0.540102,0.301608,-0.617823,0.999894,0.561304,0.252765,-0.577177,0.999771,0.575891,...,0.003934,0,0.586375,0.249291,0.014290,0,0.590373,0.244145,0.014856,0


In [51]:
df[df['class']=='Normal']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.466250,0.331764,-0.907347,0.999906,0.488859,0.275133,-0.851194,0.999823,0.501955,...,0.001799,0,0.519976,0.273510,0.020471,0,0.524371,0.268968,0.021337,0
1,Normal,0.470933,0.328804,-0.778751,0.999902,0.490143,0.273135,-0.720188,0.999819,0.502533,...,0.002995,0,0.525391,0.273477,0.023576,0,0.529398,0.270235,0.024468,0
2,Normal,0.473655,0.327398,-0.766039,0.999898,0.490900,0.272356,-0.707355,0.999816,0.502857,...,0.002160,0,0.524868,0.273799,0.021655,0,0.528982,0.270436,0.022443,0
3,Normal,0.475893,0.325781,-0.764224,0.999896,0.492181,0.271684,-0.705913,0.999815,0.503827,...,0.003089,0,0.524431,0.274338,0.022729,0,0.528567,0.270888,0.023579,0
4,Normal,0.476316,0.324335,-0.698947,0.999903,0.492545,0.270901,-0.643586,0.999828,0.504012,...,0.003550,0,0.522296,0.274781,0.022839,0,0.526642,0.270979,0.023696,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Normal,0.461352,0.136900,-0.574556,0.999807,0.468961,0.111087,-0.548456,0.999576,0.474830,...,-0.000036,0,0.490685,0.114650,0.005235,0,0.492731,0.111651,0.005377,0
286,Normal,0.478395,0.116605,-0.568166,0.999825,0.487579,0.092118,-0.540122,0.999616,0.494293,...,0.000551,0,0.515690,0.089336,0.008127,0,0.517680,0.086542,0.008414,0
287,Normal,0.517985,0.096096,-0.530836,0.999840,0.528093,0.071382,-0.511509,0.999650,0.535316,...,0.000559,0,0.550757,0.064101,0.009792,0,0.552851,0.061340,0.010242,0
288,Normal,0.547569,0.083605,-0.527620,0.999854,0.554365,0.059128,-0.496080,0.999681,0.560227,...,0.001853,0,0.570215,0.046391,0.012356,0,0.572216,0.044333,0.012839,0


In [52]:
df[df['class']=='Heart Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
290,Heart Pain,0.424330,0.284270,-0.781484,0.999926,0.440791,0.242940,-0.726478,0.999886,0.452241,...,0.004587,0,0.456013,0.238460,0.017930,0,0.459639,0.234844,0.018650,0
291,Heart Pain,0.424646,0.282075,-0.686610,0.999930,0.440889,0.240783,-0.638121,0.999887,0.452375,...,0.003736,0,0.457006,0.236550,0.017353,0,0.460560,0.233772,0.017991,0
292,Heart Pain,0.425431,0.279666,-0.679403,0.999932,0.441228,0.238658,-0.632693,0.999886,0.452743,...,0.003871,0,0.458188,0.236506,0.017996,0,0.461756,0.233614,0.018650,0
293,Heart Pain,0.425795,0.276422,-0.687483,0.999936,0.441324,0.236395,-0.639071,0.999887,0.452841,...,0.003659,0,0.458286,0.236288,0.017536,0,0.461841,0.233451,0.018162,0
294,Heart Pain,0.427420,0.273330,-0.657744,0.999939,0.442540,0.233989,-0.612027,0.999890,0.454376,...,0.003321,0,0.460228,0.233989,0.016921,0,0.463813,0.231225,0.017487,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,Heart Pain,0.603031,0.240310,-0.133693,0.999856,0.603485,0.217901,-0.105967,0.999742,0.604112,...,0.001849,0,0.615003,0.219073,0.018438,0,0.615705,0.217582,0.019649,0
682,Heart Pain,0.630482,0.239976,-0.098386,0.999840,0.630348,0.217015,-0.067377,0.999725,0.630086,...,0.002703,0,0.650602,0.225407,0.021623,0,0.652348,0.223076,0.023172,0
683,Heart Pain,0.660341,0.246722,-0.093042,0.999824,0.663124,0.226212,-0.062141,0.999707,0.662878,...,0.001548,0,0.680929,0.203546,0.016190,0,0.682565,0.200937,0.017109,0
684,Heart Pain,0.784059,0.248553,-0.158974,0.999406,0.790010,0.231295,-0.129459,0.999049,0.793062,...,0.000372,0,0.786331,0.217054,0.009484,0,0.788066,0.215969,0.009906,0


In [53]:
df[df['class']=='Head Ache']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
686,Head Ache,0.515609,0.395662,-0.632636,0.999890,0.538755,0.345018,-0.584433,0.999633,0.551396,...,0.002985,0,0.547740,0.335796,0.013607,0,0.551927,0.331043,0.013827,0
687,Head Ache,0.514230,0.395656,-0.589150,0.999892,0.535725,0.345042,-0.542840,0.999646,0.548593,...,0.002409,0,0.550311,0.333489,0.014746,0,0.554011,0.329656,0.014923,0
688,Head Ache,0.511434,0.398478,-0.570607,0.999898,0.531369,0.346574,-0.530287,0.999670,0.544248,...,0.001274,0,0.551123,0.335070,0.014365,0,0.554888,0.331617,0.014512,0
689,Head Ache,0.511637,0.398460,-0.566820,0.999906,0.531509,0.346735,-0.521648,0.999698,0.544387,...,0.000808,0,0.553236,0.336466,0.014346,0,0.556977,0.333208,0.014497,0
690,Head Ache,0.511749,0.398444,-0.562915,0.999914,0.531525,0.346891,-0.515969,0.999723,0.544422,...,0.000574,0,0.551875,0.336318,0.011803,0,0.555699,0.332877,0.011806,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,Head Ache,0.472607,0.402065,-0.612190,0.999983,0.495212,0.361007,-0.558767,0.999948,0.506906,...,0.002328,0,0.507305,0.372956,0.009877,0,0.511169,0.368582,0.010217,0
850,Head Ache,0.472622,0.400660,-0.623391,0.999982,0.495010,0.359597,-0.572165,0.999949,0.506808,...,0.003713,0,0.506880,0.368046,0.011245,0,0.510711,0.363660,0.011628,0
851,Head Ache,0.472594,0.400186,-0.627544,0.999982,0.494690,0.358359,-0.576330,0.999950,0.506652,...,0.001690,0,0.504898,0.360793,0.009553,0,0.508477,0.357323,0.009715,0
852,Head Ache,0.471756,0.399381,-0.623496,0.999982,0.492950,0.357761,-0.571674,0.999951,0.505348,...,0.001678,0,0.504151,0.361304,0.008693,0,0.507707,0.358116,0.008752,0


In [54]:
df[df['class']=='Hand Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
854,Hand Pain,0.550547,0.285351,-0.427041,0.999957,0.571911,0.249694,-0.384340,0.999904,0.584645,...,0.001893,0,0.589381,0.259454,0.012502,0,0.594007,0.256532,0.012882,0
855,Hand Pain,0.550522,0.286058,-0.375691,0.999933,0.571976,0.249767,-0.324520,0.999855,0.584561,...,0.000472,0,0.589265,0.258622,0.010921,0,0.593525,0.256305,0.011096,0
856,Hand Pain,0.549957,0.286364,-0.381702,0.999912,0.571949,0.249760,-0.331529,0.999809,0.584282,...,0.000136,0,0.588666,0.258330,0.010028,0,0.592937,0.256111,0.010140,0
857,Hand Pain,0.548073,0.286418,-0.387121,0.999895,0.571557,0.249754,-0.337685,0.999778,0.583620,...,0.000892,0,0.588609,0.257080,0.010274,0,0.592807,0.254923,0.010339,0
858,Hand Pain,0.546696,0.286729,-0.410890,0.999887,0.571091,0.249759,-0.361004,0.999752,0.582951,...,0.000485,0,0.589062,0.257131,0.010461,0,0.593196,0.255234,0.010532,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Hand Pain,0.539917,0.300630,-0.519261,0.999859,0.560269,0.252916,-0.474520,0.999697,0.574547,...,0.002297,0,0.581374,0.257538,0.012183,0,0.585121,0.252797,0.012560,0
980,Hand Pain,0.538935,0.301239,-0.503000,0.999870,0.559504,0.252953,-0.459029,0.999721,0.574010,...,0.002378,0,0.582757,0.257123,0.012633,0,0.586475,0.252842,0.013027,0
981,Hand Pain,0.539426,0.301248,-0.581417,0.999878,0.560021,0.252938,-0.533026,0.999737,0.574460,...,0.003969,0,0.584337,0.254576,0.014584,0,0.588096,0.249696,0.015153,0
982,Hand Pain,0.539693,0.301557,-0.641233,0.999886,0.560967,0.252967,-0.595796,0.999753,0.575460,...,0.003809,0,0.585066,0.252302,0.014961,0,0.588834,0.247320,0.015571,0


In [55]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [57]:
y_test

166        Normal
392    Heart Pain
40         Normal
118        Normal
953     Hand Pain
          ...    
896     Hand Pain
218        Normal
566    Heart Pain
170        Normal
697     Head Ache
Name: class, Length: 296, dtype: object

## Classification Model

In [58]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [61]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=10000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [62]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [63]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=10000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [64]:
fit_models['rc'].predict(X_test)

array(['Normal', 'Heart Pain', 'Normal', 'Normal', 'Hand Pain',
       'Hand Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Head Ache', 'Normal', 'Normal', 'Normal',
       'Hand Pain', 'Normal', 'Heart Pain', 'Normal', 'Heart Pain',
       'Normal', 'Hand Pain', 'Normal', 'Hand Pain', 'Heart Pain',
       'Normal', 'Hand Pain', 'Hand Pain', 'Heart Pain', 'Hand Pain',
       'Heart Pain', 'Normal', 'Hand Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Head Ache',
       'Head Ache', 'Normal', 'Heart Pain', 'Heart Pain', 'Hand Pain',
       'Head Ache', 'Heart Pain', 'Normal', 'Heart Pain', 'Head Ache',
       'Heart Pain', 'Heart Pain', 'Hand Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Head Ache', 'Normal', 'Heart Pain', 'Hand Pain',
       'Normal', 'Heart Pain', 'Head Ache', 'Normal', 'Heart Pain',
       'Normal', 'Normal', 'Normal', 'Hand Pain', 'Heart Pain',
       'Heart Pain', 'Hand Pain', 'Hea

## Dump model

In [65]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [66]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9932432432432432
rc 0.9932432432432432
rf 0.9966216216216216
gb 0.9966216216216216


In [67]:
fit_models['rf'].predict(X_test)

array(['Normal', 'Heart Pain', 'Normal', 'Normal', 'Hand Pain',
       'Hand Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Head Ache', 'Normal', 'Normal', 'Normal',
       'Hand Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Normal', 'Hand Pain', 'Normal', 'Hand Pain', 'Heart Pain',
       'Normal', 'Hand Pain', 'Hand Pain', 'Heart Pain', 'Hand Pain',
       'Heart Pain', 'Normal', 'Hand Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Head Ache',
       'Head Ache', 'Normal', 'Heart Pain', 'Heart Pain', 'Hand Pain',
       'Head Ache', 'Heart Pain', 'Normal', 'Heart Pain', 'Head Ache',
       'Heart Pain', 'Heart Pain', 'Hand Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Head Ache', 'Normal', 'Heart Pain', 'Hand Pain',
       'Normal', 'Heart Pain', 'Head Ache', 'Normal', 'Heart Pain',
       'Normal', 'Normal', 'Normal', 'Hand Pain', 'Heart Pain',
       'Heart Pain', 'Hand Pain', 

In [68]:
y_test

166        Normal
392    Heart Pain
40         Normal
118        Normal
953     Hand Pain
          ...    
896     Hand Pain
218        Normal
566    Heart Pain
170        Normal
697     Head Ache
Name: class, Length: 296, dtype: object

In [69]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detection

In [70]:
import pyttsx3
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [71]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [72]:
# feature names
fnames = []
for val in range(1, num_coords+1):
    fnames += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [73]:
engine = pyttsx3.init()
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row], columns=fnames)
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Audio Alert
            if body_language_class != "Normal":
                engine.say(str(body_language_class)+"detected")
                engine.runAndWait()
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()